# torch的基本操作

In [1]:
import numpy as np
import torch

## 1 创建一个tensor
    1.从列表创建
    2.从np.array创建

In [2]:
# t1列表， t2 np.array
t1 = [
    [1, 2, 3],
    [4, 5, 6]
]
t2 = np.array([
    [7, 8, 9],
    [10, 11, 12]
])

In [3]:
# from list
t1 = torch.Tensor(t1)
print(t1)


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]



In [4]:
# from np.array
t2 = torch.from_numpy(t2).float()
print(t2)


  7   8   9
 10  11  12
[torch.FloatTensor of size 2x3]



##  2 拼接
    这里指定的维度参数是dim，而不是axis

In [5]:
t_concat0= torch.cat([t1, t2], dim=0)
print(t_concat0)


  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.FloatTensor of size 4x3]



In [6]:
t_concat1 = torch.cat([t1, t2], dim=1)
print(t_concat1)


  1   2   3   7   8   9
  4   5   6  10  11  12
[torch.FloatTensor of size 2x6]



## 3 切片
    3.1 可以直接用类似python的方法切
    3.2 可以用chunk, 指定chunk的个数chunk，维度, （这里的chunks如果不能整除的话，会自动按小于chunks的最大可整除数目切分， 例如：维度是6,chunks为5,会自动按3切）
    3.3 可以用index_select在特定维度取出特定数据, 注意index必须要传入LongTensor，不能用列表
    3.4 split ,需要设置split_size（每块有多少数据），如果当前的未切分数据比split_size小，就直接返回当前数据，例如一共6条，split_size=5，返回两块数据，第一块5条，第二块1条

In [7]:
# 直接切
print(t_concat0[:, 1:3])


  2   3
  5   6
  8   9
 11  12
[torch.FloatTensor of size 4x2]



In [8]:
# 等分chunk切
print(torch.chunk(t_concat1, chunks=5, dim=1))

(
 1  2
 4  5
[torch.FloatTensor of size 2x2]
, 
  3   7
  6  10
[torch.FloatTensor of size 2x2]
, 
  8   9
 11  12
[torch.FloatTensor of size 2x2]
)


In [9]:
# index_select切
print(torch.index_select(t_concat1, dim=1, index=torch.LongTensor([1, 3, 5])))


  2   7   9
  5  10  12
[torch.FloatTensor of size 2x3]



In [10]:
# split切
print(torch.split(t_concat1, split_size=5, dim=1))

(
  1   2   3   7   8
  4   5   6  10  11
[torch.FloatTensor of size 2x5]
, 
  9
 12
[torch.FloatTensor of size 2x1]
)


## 4 增加/减少维度
    1.增加维度用unsqueeze
    2.减小维度用squeeze, 如果不指定dim，就把所有为1的维度压缩，如果指定dim就只压缩指定的

In [11]:
# 增加维度
t_unsq = torch.unsqueeze(t_concat0, dim=0)
print(t_unsq)
t_unsq = torch.unsqueeze(t_unsq, dim=2)
print(t_unsq)


(0 ,.,.) = 
   1   2   3
   4   5   6
   7   8   9
  10  11  12
[torch.FloatTensor of size 1x4x3]


(0 ,0 ,.,.) = 
   1   2   3

(0 ,1 ,.,.) = 
   4   5   6

(0 ,2 ,.,.) = 
   7   8   9

(0 ,3 ,.,.) = 
  10  11  12
[torch.FloatTensor of size 1x4x1x3]



In [12]:
# 减小维度
print(torch.squeeze(t_unsq))
print(torch.squeeze(t_unsq, dim=2))


  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.FloatTensor of size 4x3]


(0 ,.,.) = 
   1   2   3
   4   5   6
   7   8   9
  10  11  12
[torch.FloatTensor of size 1x4x3]



## 5 转换维度
    1. permute, 将重新排列的维度写在括号里就可以了

In [13]:
print(t1.permute(1, 0))


 1  4
 2  5
 3  6
[torch.FloatTensor of size 3x2]



## 6 stack
    会增加维度，在dim处加，然后在dim处cat,  默认dim=0
    stack([a, b], dim) 相当于下面两步的集合：
        1. a = torch.unsqueeze(a, dim), b = torch.unsqueeze(b, dim)
        2. torch.cat([a, b], dim)

In [14]:
print(torch.stack([t1, t2], dim=0))
print(torch.cat([t1, t2], dim=0))


(0 ,.,.) = 
   1   2   3
   4   5   6

(1 ,.,.) = 
   7   8   9
  10  11  12
[torch.FloatTensor of size 2x2x3]


  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.FloatTensor of size 4x3]



In [15]:
print(torch.stack([t1, t2], dim=1))
print(torch.cat([t1, t2], dim=1))


(0 ,.,.) = 
   1   2   3
   7   8   9

(1 ,.,.) = 
   4   5   6
  10  11  12
[torch.FloatTensor of size 2x2x3]


  1   2   3   7   8   9
  4   5   6  10  11  12
[torch.FloatTensor of size 2x6]



In [16]:
print(torch.stack([t1, t2], dim=2))


(0 ,.,.) = 
   1   7
   2   8
   3   9

(1 ,.,.) = 
   4  10
   5  11
   6  12
[torch.FloatTensor of size 2x3x2]



In [17]:
# 验证一下，stack是不是如上面两步说的
t11 = torch.unsqueeze(t1, 2)
t22 = torch.unsqueeze(t2, 2)
print(torch.cat([t11, t22], 2))


(0 ,.,.) = 
   1   7
   2   8
   3   9

(1 ,.,.) = 
   4  10
   5  11
   6  12
[torch.FloatTensor of size 2x3x2]

